In [1]:
def allow_direct_imports_from(dirname):
    import sys
    if dirname not in sys.path:
        sys.path.append(dirname)
        
allow_direct_imports_from('automl/efficientdet')

import os, shutil, matplotlib.pyplot as plt, tensorflow as tf
import tf2.infer_lib as infer_lib, util, custom_callbacks, train_data_generator, attacker

In [2]:
MODEL = 'efficientdet-lite4'

def main(download_model=False):
    log_dir = util.ensure_empty_dir('log_dir')
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)

    victim_model = attacker.get_victim_model(download_model)
    config_override = {'nms_configs': {'iou_thresh': .5, 'score_thresh': .5}}
    model = attacker.PatchAttacker(victim_model, 'save_dir_1/patch_20.png',
                                   patch_loss_multiplier=0., config_override=config_override,
                                   visualize_freq=200)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), run_eagerly=False)

    datasets: dict = train_data_generator.partition(model.config, 'downloaded_images', 'labels',
                                                               batch_size=12, shuffle=True)

    train_ds = datasets['train']['dataset']
    val_ds = datasets['val']['dataset']
    train_len = datasets['train']['length']
    val_len = datasets['val']['length']
    tb_callback = custom_callbacks.TensorboardCallback(log_dir, write_graph=True, write_steps_per_second=True)
    model.tb = tb_callback

    save_dir = util.ensure_empty_dir('save_dir')
    save_file = 'patch_{epoch:02d}_{val_mean_asr:.4f}.tiff'
    history = model.fit(train_ds, validation_data=val_ds, epochs=50, steps_per_epoch=train_len,
                        initial_epoch=20,
                        validation_steps=val_len,
                        callbacks=[tb_callback,
                                   tf.keras.callbacks.ModelCheckpoint(os.path.join(save_dir, save_file),
                                                                      monitor='val_mean_asr',
                                                                      verbose=1,
                                                                      save_best_only=False,
                                                                      save_weights_only=True,
                                                                      mode='auto',
                                                                      save_freq='epoch',
                                                                      options=None,
                                                                      initial_value_threshold=None,
                                                                      )
                                   ])
    patch = model.get_patch().astype('uint8')
    plt.imshow(patch)
    plt.show()

In [ ]:
main()

Using model in efficientdet-lite4
/tmp/__autograph_generated_file30xqcx1w.py:23: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  ag__.for_stmt(ag__.ld(self).updates, None, loop_body, get_state, set_state, (), {'iterate_names': 'u'})
/phyadv/automl/efficientdet/utils.py:255: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  for u in self.updates:
filtering dataset by label constraints...
done. data size is 64115
training on 51292 images, validating on 6411


Epoch 21/50


Instructions for updating:
Use fn_output_signature instead


Instructions for updating:
Use fn_output_signature instead


4275/4275 [==============================] - ETA: 0s - loss: 6.4709 - mean_asr: 0.5880WARNING:tensorflow:Using a while_loop for converting NonMaxSuppressionV5


...

4077/4275 [===========================>..] - ETA: 2:20 - loss: 6.1972 - mean_asr: 0.6235

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4275/4275 [==============================] - ETA: 0s - loss: 6.0736 - mean_asr: 0.6375
Epoch 29: saving model to save_dir/patch_29_0.6632.tiff
4275/4275 [==============================] - 3291s 770ms/step - loss: 6.0734 - mean_asr: 0.6375 - val_loss: 7.6736 - val_mean_asr: 0.6632
Epoch 30/50
4275/4275 [==============================] - ETA: 0s - loss: 6.0392 - mean_asr: 0.6411
Epoch 30: saving model to save_dir/patch_30_0.6708.tiff
4275/4275 [==============================] - 3292s 770ms/step - loss: 6.0391 - mean_asr: 0.6411 - val_loss: 6.5772 - val_mean_asr: 0.6708
Epoch 31/50
4275/4275 [==============================] - ETA: 0s - loss: 6.0001 - mean_asr: 0.6451
Epoch 31: saving model to save_dir/patch_31_0.6700.tiff
4275/4275 [==============================] - 3293s 770ms/step - loss: 6.0001 - mean_asr: 0.6451 - val_loss: 5.1116 - val_mean_asr: 0.6700
Epoch 32/50
4275/4275 [==============================] - ETA: 0s - loss: 5.9400 - mean_asr: 0.6521
Epoch 33: saving model to save_dir

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



4275/4275 [==============================] - ETA: 0s - loss: 5.7574 - mean_asr: 0.6686
Epoch 40: saving model to save_dir/patch_40_0.6955.tiff
4275/4275 [==============================] - 3287s 769ms/step - loss: 5.7574 - mean_asr: 0.6686 - val_loss: 6.8655 - val_mean_asr: 0.6955
Epoch 41/50
 884/4275 [=====>........................] - ETA: 39:51 - loss: 5.7204 - mean_asr: 0.6725